# Arts Council Funding by Org Type

This script combines funding information from the Arts Council Investment Programme, National Lottery Project Grants and Companies House data to explore the relationship between organisation size and the amount of funding received. 

Note: As a proxy for organisation size, we have used the accounting category from Companies House. Accounts categories offer an indication of the size and type of organisation. For definitions of account types, <a href="https://find-and-update.company-information.service.gov.uk/guides/accounts/chooser">see the Companies House definitions</a>.  



In [1]:
# Import libraries

import pandas as pd
from pathlib import Path
import os

In [1]:
# Define paths to the files we need

OUT_DIR = Path('../src/culture-sector/funding/_data')
OUT_DIR.mkdir(exist_ok=True, parents=True)

ORG_DATA = Path('../data/culture_landscape.csv')
INVESTMENT_DATA = Path('../raw/arts-council-investment-programme.csv')
GRANTS_DATA = Path('../raw/arts-council-project-grants.csv')

CORRECTIONS = Path('../raw/landscape-map-corrections.csv')

NameError: name 'Path' is not defined

In [10]:
# Read in the files we need

org_data = pd.read_csv(ORG_DATA)
invest_data = pd.read_csv(INVESTMENT_DATA)
grants_data = pd.read_csv(GRANTS_DATA)
corrections_data = pd.read_csv(CORRECTIONS)

In [12]:
# TODO: We may wish to present the number of UNKNOWN categories. 
# TODO: Fuzzy match and include list of corrections. 

# Drop records where the accounts catgory is unknown. 
acc_cat = org_data.dropna(subset=['accounts_category'])

# Merge in Project Grants data
funding_amounts = (
    acc_cat.merge(grants_data[['Recipient', 'Award amount']], 
                          left_on='organisation', 
                          right_on='Recipient', 
                          how='left')
    .rename(columns={'Award amount': 'project_grant_amount'})
    .drop(columns='Recipient')
) 

# Merge in investment grants data
funding_amounts = (
    funding_amounts.merge(invest_data[['Applicant Name', '2023-26 Annual Funding (Offered 4 Nov 2022)']], 
                          left_on='organisation', 
                          right_on='Applicant Name', 
                          how='left')
    .rename(columns={'2023-26 Annual Funding (Offered 4 Nov 2022)': 'investment_amount'})
    .drop(columns='Applicant Name')
)


,organisation,LAD24CD,LAD24NM,IPSO,NPO,Project Grant,latitude,longitude,company_category,accounts_category,company_number,company_status,dissolution_date,incorporation_date,post_town,postcode,sic_code,uri,project_grant_amount,investment_amount
0,Action for Children,E06000057,Northumberland,NaN,NaN,True,NaN,NaN,"PRI/LBG/NSC (Private, Limited by guarantee, no...",GROUP,04764232,Active,NaN,2003-05-14,WATFORD,WD18 8AG,['87900 - Other residential care activities n....,http://business.data.gov.uk/id/company/04764232,28000.0,NaN
1,Alphabetti Theatre,E08000021,Newcastle upon Tyne,NaN,NaN,True,NaN,NaN,Charitable Incorporated Organisation,NO ACCOUNTS FILED,CE006183,Active,NaN,2016-04-08,NaN,NaN,['None Supplied'],http://business.data.gov.uk/id/company/CE006183,90000.0,NaN
2,Alphabetti Theatre,E08000021,Newcastle upon Tyne,NaN,NaN,True,NaN,NaN,Charitable Incorporated Organisation,NO ACCOUNTS FILED,CE006183,Active,NaN,2016-04-08,NaN,NaN,['None Supplied'],http://business.data.gov.uk/id/company/CE006183,98000.0,NaN
3,Alphabetti Theatre,E08000021,Newcastle upon Tyne,NaN,NaN,True,NaN,NaN,Charitable Incorporated Organisation,NO ACCOUNTS FILED,CE006183,Active,NaN,2016-04-08,NaN,NaN,['None Supplied'],http://business.data.gov.uk/id/company/CE006183,37500.0,NaN
4,Alphabetti Theatre,E08000021,Newcastle upon Tyne,NaN,NaN,True,NaN,NaN,Charitable Incorporated Organisation,NO ACCOUNTS FILED,CE006183,Active,NaN,2016-04-08,NaN,NaN,['None Supplied'],http://business.data.gov.uk/id/company/CE006183,14990.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,Woodhorn Charitable Trust,E06000057,Northumberland,NaN,True,NaN,NaN,NaN,"PRI/LBG/NSC (Private, Limited by guarantee, no...",GROUP,06893854,Active,NaN,2009-05-01,ASHINGTON,NE63 9YF,['91020 - Museums activities'\n '91030 - Opera...,http://business.data.gov.uk/id/company/06893854,NaN,346680.0
171,Workie Ticket Theatre CIC,E08000022,North Tyneside,NaN,NaN,True,NaN,NaN,Community Interest Company,TOTAL EXEMPTION FULL,11780194,Active,NaN,2019-01-22,NEWCASTLE UPON TYNE,NE27 0BD,['85520 - Cultural education' '90010 - Perform...,http://business.data.gov.uk/id/company/11780194,14610.0,NaN
172,YMCA North Tyneside,E08000022,North Tyneside,NaN,NaN,True,NaN,NaN,"PRI/LBG/NSC (Private, Limited by guarantee, no...",GROUP,02703063,Active,NaN,1992-04-02,TYNE AND WEAR,NE29 0AB,['96090 - Other service activities n.e.c.'],http://business.data.gov.uk/id/company/02703063,6918.0,NaN
173,tiny dragon Productions,E08000021,Newcastle upon Tyne,NaN,NaN,True,NaN,NaN,"PRI/LBG/NSC (Private, Limited by guarantee, no...",TOTAL EXEMPTION FULL,13816200,Active,NaN,2021-12-23,NEWCASTLE UPON TYNE,NE1 1LE,['90010 - Performing arts' '90030 - Artistic c...,http://business.data.gov.uk/id/company/13816200,30000.0,NaN


In [11]:
funding_amounts['total_funding'] = (funding_amounts['project_grant_amount'].fillna(0) + 
                                    funding_amounts['investment_amount'].fillna(0)).astype(int)

total_funding = funding_amounts[['organisation', 'accounts_category', 'total_funding']]

,organisation,accounts_category,total_funding
0,Action for Children,GROUP,28000
1,Alphabetti Theatre,NO ACCOUNTS FILED,90000
2,Alphabetti Theatre,NO ACCOUNTS FILED,98000
3,Alphabetti Theatre,NO ACCOUNTS FILED,37500
4,Alphabetti Theatre,NO ACCOUNTS FILED,14990
...,...,...,...
170,Woodhorn Charitable Trust,GROUP,346680
171,Workie Ticket Theatre CIC,TOTAL EXEMPTION FULL,14610
172,YMCA North Tyneside,GROUP,6918
173,tiny dragon Productions,TOTAL EXEMPTION FULL,30000


In [7]:
# Group by accounting category and calculate the average funding amount

average_grant_award_by_acc_cat = (
    total_funding.groupby('accounts_category')['total_funding'].mean()
    .astype(int)
    .reset_index()
    .to_csv(os.path.join(OUT_DIR, 'grant_by_org_size.csv'), index=False)
)